# Setup

In [ ]:
%pip install mlflow scikit-learn==1.5.0

In [ ]:
import digitalhub as dh

project = dh.get_or_create_project("demo-mlflow")

# Train the model

In [ ]:
%%writefile train-model.py

from digitalhub_runtime_python import handler

from digitalhub import from_mlflow_run
import mlflow

from sklearn import datasets, svm
from sklearn.model_selection import GridSearchCV

@handler()
def train(project):
    mlflow.sklearn.autolog(log_datasets=True)

    iris = datasets.load_iris()
    parameters = {"kernel": ("linear", "rbf"), "C": [1, 10]}
    svc = svm.SVC()
    clf = GridSearchCV(svc, parameters)

    clf.fit(iris.data, iris.target)
    run_id = mlflow.last_active_run().info.run_id

    # utility to map mlflow run artifacts to model metadata
    model_params = from_mlflow_run(run_id)

    project.log_model(
        name="model-mlflow",
        kind="mlflow",
        **model_params
)

In [ ]:
train_fn = project.new_function(name="train",
                                kind="python",
                                python_version="PYTHON3_10",
                                code_src="train-model.py",
                                handler="train",
                                requirements=["scikit-learn==1.5.0", "mlflow==2.15.1"])

In [ ]:
train_run = train_fn.run(action="job", local_execution=True)

In [ ]:
model = project.get_model("model-mlflow")
model.spec.path

# Deploy the model

In [ ]:
func = project.new_function(name="serve_mlflowmodel",
                            kind="mlflowserve",
                            model_name="mlflow_model",
                            path=model.spec.path + 'model/')

serve_run = func.run(action="serve")

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()
data = iris.data[0:2].tolist()
json={
    "inputs": [
        {
        "name": "input-0",
        "shape": [-1, 4],
        "datatype": "FP64",
        "data": data
        }
    ]
}

# Test the endpoint

You can check the status of the model with the following. When the attribute *service* appears, the model is ready to be used.

In [ ]:
serve_run.refresh().status.service

In [ ]:
serve_run.invoke(model_name="mlflow_model", json=json).json()